In [1]:
from qdrant_client import QdrantClient, models
from fastembed import TextEmbedding

/home/codespace/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
qd_client = QdrantClient('http://localhost:6333')

In [3]:
import json

with open('documents.json', 'r') as documents_file:
    documents_raw = json.load(documents_file)

documents = []

for course_dict in documents_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [8]:
model_handle = "Qdrant/bm25"
collection_name = "zoomcamp-parse"
# EMBEDDING_DIMENSIONALITY = 512 For BM, we do not need to define the size

In [5]:
qd_client.create_collection(
    collection_name=collection_name,
    sparse_vectors_config={
        "bm25":models.SparseVectorParams(
            modifier=models.Modifier.IDF
        )
    }
)

True

In [10]:
import uuid

points = []
for doc in documents:
    point = models.PointStruct(
        id=uuid.uuid4().hex,
        vector={
            'bm25': models.Document(
                text=doc['text'],
                model=model_handle
            )
        },
        payload=doc
    )
    points.append(point)

In [11]:
qd_client.upsert(collection_name=collection_name, points=points)

Fetching 18 files: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 24.07it/s]


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [15]:
def search(query: str, limit:int = 1) -> list[models.ScoredPoint]:
    results = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model=model_handle
        ),
        using='bm25',
        limit=limit,
        with_payload=True
    )
    return results.points

In [16]:
results = search('pandas', limit=1)
print(results[0].payload['text'])

You can use round() function or f-strings
round(number, 4)  - this will round number up to 4 decimal places
print(f'Average mark for the Homework is {avg:.3f}') - using F string
Also there is pandas.Series. round idf you need to round values in the whole Series
Please check the documentation
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.round.html#pandas.Series.round
Added by Olga Rudakova
